In [1]:
from mmdet.apis import init_detector, inference_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed

import glob as _glob
import os
import json

In [2]:
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches

In [3]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('mmdetection/configs/yolof/yolof_r50_c5_8x8_1x_coco.py')

In [4]:
@DATASETS.register_module()
class Auto_drive_dataset(CustomDataset):
    CLASSES=('car','pedestrian','traffic sign', 'motorcycle', 'bus','truck','bicycle','traffic light','special vehicle', 'non')


    def load_annotations(self, ann_file):
        
        CLASSES_dict = {'car' : 0 , 'pedestrian':1, 'traffic sign':2, 'motorcycle':3, 'bus':4,'truck':5 ,'bicycle':6 ,'traffic light':7,'special vehicle':8,'non':9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
        
        data_infos = []
        
        for idx,img in enumerate(image_list):
            json_data = {}
            with open(img, "r") as json_file:
                json_data = json.load(json_file)
            # 수정 prefix 제대로 적용안되서 문자열 그대로 입력 
            filename = '2D_BB'+'/'+json_data['Source_Image_Info']['Img_path'][0:9]+'/'+json_data['Source_Image_Info']['Img_path']+'/'+json_data['Source_Image_Info']['Img_name']

            height, width = json_data['Source_Image_Info']['Resolution']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for idx in range(len(json_data['Annotation'])):
                gt_labels.append(CLASSES_dict[json_data['Annotation'][idx]['Label']])
                gt_bboxes.append(json_data['Annotation'][idx]['Coordinate'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(image_list)+' load annotations END!')
            
        
        
        return data_infos
    

In [5]:
cfg.dataset_type  = 'Auto_drive_dataset'
cfg.data_root = ''
cfg.img_scale = (384, 384)
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=cfg.img_scale,
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Pad', size=cfg.img_scale, pad_val=114.0),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]


#추가수정 ( default class 80개라서 에러발생)
cfg.model.bbox_head.num_classes=10

cfg.data.test.pipeline = cfg.test_pipeline

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')


Config:
dataset_type = 'Auto_drive_dataset'
data_root = ''
img_norm_cfg = dict(
    mean=[103.53, 116.28, 123.675], std=[1.0, 1.0, 1.0], to_rgb=False)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='RandomShift', shift_ratio=0.5, max_shift_px=32),
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(384, 384),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Pad', size=(384, 38

In [6]:
# Build the detector
#model = init_detector(config, checkpoint, device='cuda:0')
checkpoint='checkpoints/epoch_4.pth'
model = init_detector(cfg,checkpoint, device='cuda:0')

load checkpoint from local path: checkpoints/epoch_4.pth


In [7]:
import json
from mmdet.core.bbox import BboxOverlaps2D


def IoU(box1, box2):
    # box = (x1, y1, x2, y2)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # obtain x1, y1, x2, y2 of the intersection
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # compute the width and height of the intersection
    w = max(0, x2 - x1 + 1)
    h = max(0, y2 - y1 + 1)

    inter = w * h
    iou = inter / (box1_area + box2_area - inter)
    return iou

# valdiation 대상 목록 load
ls=mmcv.list_from_file('splits/val.txt')

CLASSES_dict = {0:'car'  , 1:'pedestrian', 2:'traffic sign', 3:'motorcycle', 4:'bus',5:'truck' ,6:'bicycle' ,7:'traffic light',8:'special vehicle',9:'non'}

anno_data = {}


########################################
# mAP 계산 대상 txt파일로 생성 
######################################

# IOU 측정용 데이터 생성 part
# 저장된 output 양식의 클래스명에 ' '가 있으면 안되서 ' '는 '_' 로 수정
for idx, data in enumerate(ls):
    json_data = {}
    with open(data, "r") as json_file:
        json_data = json.load(json_file)
    
    imgpath='2D_BB/'+json_data['Source_Image_Info']['Img_path'][0:9]+'/'+json_data['Source_Image_Info']['Img_path']+'/'+json_data['Source_Image_Info']['Img_name']
    
    gt_bboxes = []
    gt_labels = []

    results = inference_detector(model, imgpath)
    
    #eval_map(result,anno_data)
    f = open("output_for_val/detection_result/"+json_data['Source_Image_Info']['Img_name'][0:-4]+'.txt','a')
    
    # 예측값 write
    for iidx, result in enumerate(results):
        for wr_t in result:
                f.write(CLASSES_dict[iidx].replace(' ','_')+' '+str(wr_t[4])+' '+str(wr_t[0])+' '+str(wr_t[1])+' '+str(wr_t[2])+' '+str(wr_t[3])+'\n')

    f.close()
    
    f = open("output_for_val/ground_truth/"+json_data['Source_Image_Info']['Img_name'][0:-4]+'.txt','a')
    
    # grundtruth write
    for anno in json_data['Annotation']:
        f.write(anno['Label'].replace(' ','_')+' '+str(anno['Coordinate'][0])+' '+str(anno['Coordinate'][1])+' '+str(anno['Coordinate'][2])+' '+str(anno['Coordinate'][3])+'\n')
    f.close()
    
    if idx%1000==0:
        print( str(idx)+'/'+str(len(ls))+' END!')

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


0/9519 END!
1000/9519 END!
2000/9519 END!
3000/9519 END!
4000/9519 END!
5000/9519 END!
6000/9519 END!
7000/9519 END!
8000/9519 END!
9000/9519 END!


In [11]:
#  https://github.com/Cartucho/mAP/ 소스 사용 
#  default로 IOU_threshold값 0.5로 되어있음 
#  위에서 저장한 outpput text file과 groundtruth text file 기준으로 처리 
!python mAP/main.py --detection_path=/root/output_for_val/detection_result --groundtruth_path=/root/output_for_val/ground_truth 

/root/output_for_val/detection_result
/root/output_for_val/ground_truth
20.87% = bicycle AP 
mAP/main.py:721: MatplotlibDeprecationWarning: 
The set_window_title function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use manager.set_window_title or GUI-specific methods instead.
38.40% = bus AP 
34.32% = car AP 
4.75% = motorcycle AP 
1.04% = non AP 
11.19% = pedestrian AP 
38.86% = special_vehicle AP 
1.14% = traffic_light AP 
8.64% = traffic_sign AP 
36.21% = truck AP 
mAP = 19.54%
mAP/main.py:314: MatplotlibDeprecationWarning: 
The set_window_title function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use manager.set_window_title or GUI-specific methods instead.
Figure(640x480)
